In [5]:
import pandas as pd
import altair as alt
import re

# Cargar el dataset original y filtrar solo las atletas chilenas
# Read the file as a single column initially, as pd.read_csv seems to be failing to split by comma.
df = pd.read_csv('/content/Copia-de-mujeres_chilenas_1.csv', header=None)

# The DataFrame 'df' now likely has a single column (e.g., named '0').
# Each row in this column contains the full comma-separated string for that row.

# Split the single column into multiple columns using comma as a delimiter
split_data = df[0].astype(str).str.split(',', expand=True)

# The first row of split_data contains the actual column names.
# Extract and clean these column names, ensuring uniqueness
original_headers = split_data.iloc[0].astype(str).tolist()
new_columns_unique = []
seen_cols = {}
for header in original_headers:
    cleaned_header = re.sub(r'[^a-z0-9_]', '', header.lower().strip())
    if cleaned_header in seen_cols:
        seen_cols[cleaned_header] += 1
        new_columns_unique.append(f"{cleaned_header}_{seen_cols[cleaned_header]}")
    else:
        seen_cols[cleaned_header] = 0
        new_columns_unique.append(cleaned_header)

# Assign these new unique column names to the rest of the data (from the second row onwards)
df = split_data[1:] # Drop the header row from the data
df.columns = new_columns_unique # Assign the new unique cleaned column names
df = df.reset_index(drop=True) # Reset index after dropping the first row

# Check if 'country' column exists after cleaning
if 'country' not in df.columns:
    print(f"Error: Column 'country' not found. Available columns after cleaning: {df.columns.tolist()}")
    raise KeyError("The 'country' column could not be found in the DataFrame after processing the CSV file. Please ensure your CSV file contains a column representing 'country' (e.g., 'Country', 'Pais', 'Nacion') or update the code to use the correct column name.")

# Normalizar texto en la columna 'country'
df['country'] = df['country'].astype(str).str.strip().str.title()

# Filtrar solo las atletas chilenas
df = df[df['country'] == 'Chile'].copy() # Se usa .copy() para evitar SettingWithCopyWarning

# Ensure 'date' is in datetime format and create 'anio' column
# Attempt to parse 'date' column (though it likely lacks year)
df['date_parsed'] = pd.to_datetime(df['date'], errors='coerce')
df['anio'] = df['date_parsed'].dt.year

# Extract year from 'none' column if 'anio' is still NaN
def extract_year_from_text(text):
    if pd.isna(text):
        return None
    match = re.search(r'(\d{4})', str(text))
    return int(match.group(1)) if match else None

# Use the first 'none' column for year extraction
# After making column names unique, df['none'] should refer to a single column
# If it's still possible that 'none' column does not exist after renaming, add a check.
if 'none' in df.columns:
    df['anio_from_none'] = df['none'].apply(extract_year_from_text)
elif 'none_0' in df.columns:
    df['anio_from_none'] = df['none_0'].apply(extract_year_from_text)
else:
    print("Warning: Neither 'none' nor 'none_0' column found for year extraction.")
    df['anio_from_none'] = pd.NA


# Fill missing 'anio' values with years extracted from 'none' column
df['anio'] = df['anio'].fillna(df['anio_from_none'])

# Convert 'anio' to numeric, coercing any remaining errors to NaN
df['anio'] = pd.to_numeric(df['anio'], errors='coerce')

# Clean and convert 'mark' to numeric
df['mark'] = df['mark'].astype(str).str.replace(',', '.', regex=False)
df['mark'] = pd.to_numeric(df['mark'], errors='coerce')

# Eliminar filas vacías donde 'mark', 'athlete_or_team', o 'anio' son NaN
df = df.dropna(subset=['mark', 'athlete_or_team', 'anio'])

# Visualización atómica
chart = (
    alt.Chart(df)
    .mark_line(point=True, strokeWidth=2)
    .encode(
        x=alt.X('anio:O', title='Año'),
        y=alt.Y('mark:Q', title='Marca (tiempo/distancia)'),
        color=alt.Color('athlete_or_team:N', title='Atleta'),
        tooltip=['anio', 'athlete_or_team', 'event', 'mark']
    )
    .properties(
        title='Progreso de las marcas por atleta chilena (1951–2023)',
        width=750,
        height=380
    )
    .configure_title(
        font='Montserrat',
        fontSize=18,
        anchor='start',
        color='#111111'
    )
    .configure_axis(
        labelFontSize=12,
        titleFontSize=13,
        labelColor='#333333',
        titleColor='#111111'
    )
)

chart.save('vis_01.html')
chart

/tmp/ipython-input-4253433156.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_parsed'] = pd.to_datetime(df['date'], errors='coerce')


alt.Chart(...)

In [4]:
df.columns.tolist()

['gender',
 'event',
 'round',
 'heat',
 'position',
 'athlete_or_team',
 'country',
 'mark',
 'status',
 'date',
 'location',
 'members',
 'genero_detectado',
 'genero',
 'fuente',
 'anio',
 'marca',
 'nombre_atleta']

In [5]:
df[['anio', 'mark', 'athlete_or_team']].head(10)

,anio,mark,athlete_or_team


In [6]:
# limpiar location
df['location_clean'] = (
    df['location']
    .astype(str)
    .str.replace('"', '')
    .str.lower()
    .str.strip()
)

# inicializamos año
df['anio'] = None

df.loc[df['location_clean'].str.contains('buenos aires'), 'anio'] = 1951
df.loc[df['location_clean'].str.contains('mexico'), 'anio'] = 1955   # puede ser 1975
df.loc[df['location_clean'].str.contains('winnipeg'), 'anio'] = 1999
df.loc[df['location_clean'].str.contains('toronto'), 'anio'] = 2015
df.loc[df['location_clean'].str.contains('santiago'), 'anio'] = 2023

In [7]:
print(df[['location', 'anio']].head(30))

Empty DataFrame
Columns: [location, anio]
Index: []


In [8]:
df[['event','location']].head(50)

,event,location


In [9]:
print(df.columns.tolist())

['gender', 'event', 'round', 'heat', 'position', 'athlete_or_team', 'country', 'mark', 'status', 'date', 'location', 'members', 'genero_detectado', 'genero', 'fuente', 'anio', 'marca', 'nombre_atleta', 'location_clean']


In [13]:
import pandas as pd
import altair as alt
import re

# Leer archivo en una sola columna
df_raw = pd.read_csv('/content/Copia-de-mujeres_chilenas_1.csv', header=None)
df_raw.columns = ['raw']

# Separar columnas por coma
df_split = df_raw['raw'].astype(str).str.split(',', expand=True)

# La fila 0 son los encabezados reales
column_names = df_split.iloc[0].astype(str).tolist()

# Renombrar columnas limpiamente y de forma única
clean_names = []
seen = {}
for name in column_names:
    cleaned = re.sub(r'[^a-z0-9]', '', name.lower().strip())
    if cleaned in seen:
        seen[cleaned] += 1
        cleaned = f"{cleaned}_{seen[cleaned]}"
    else:
        seen[cleaned] = 0
    clean_names.append(cleaned)

# Aplicar nombres limpios
df = df_split[1:].copy()
df.columns = clean_names
df = df.reset_index(drop=True)

# --- Extraer año desde TODAS las columnas que comienzan con 'none'
none_cols = [col for col in df.columns if col.startswith("none") or col.startswith("fuente")]

def extract_year(text):
    if pd.isna(text):
        return None
    m = re.search(r'(\d{4})', str(text))
    return int(m.group(1)) if m else None

df['anio'] = None
for col in none_cols:
    df['anio'] = df['anio'].fillna(df[col].apply(extract_year))

# Limpiar marca
df['mark'] = df['mark'].astype(str)
df['mark'] = df['mark'].str.replace(r'\(.*\)', '', regex=True)
df['mark'] = df['mark'].str.replace('"','').str.replace(',','.')
df['mark'] = pd.to_numeric(df['mark'], errors='coerce')

# Filtrar Chile
df = df[df['country'].str.contains('chile', case=False, na=False)]

# Eliminar nulos, usando el nombre de columna corregido 'athleteorteam'
df = df.dropna(subset=['anio','mark','athleteorteam'])

# Visualización
chart = (
    alt.Chart(df)
    .mark_line(point=True)
    .encode(
        x=alt.X('anio:O', title='Año'),
        y=alt.Y('mark:Q', title='Marca'),
        color='athleteorteam:N', # Usando el nombre de columna corregido
        tooltip=['anio','athleteorteam','event','mark'] # Usando el nombre de columna corregido
    )
    .properties(width=700, height=400,
                title='Progreso de las marcas por atleta chilena (1951–2023)')
)

chart.save('vis_01.html')
chart

/tmp/ipython-input-1695603784.py:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['anio'] = df['anio'].fillna(df[col].apply(extract_year))


alt.Chart(...)